In [1]:
import pandas as pd
import enchant   
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import DataFrame
from openpyxl import Workbook
import xlsxwriter
import math
from collections import Counter
import os

In [2]:
def read_excel(fileName, rowCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF):
    AL = pd.read_excel(fileName)
    columns_list = AL.columns

    AL.rename(columns = {columns_list[0]:'Attendee', 
                        'Unnamed: 1':'Score','Unnamed: 2':'    1:46 PM','Unnamed: 3':'        1:55 PM','Unnamed: 4':'     2:01 PM',
                        'Unnamed: 5':'       2:41 PM'},inplace=True)
    AL= AL.drop(AL.index [  0:4  ] )
    AL= AL.drop(AL.index [  0:2  ] )
    AL.drop(AL.tail(1).index,inplace=True)
    print(AL)
    master_count=0
    for i in ML.iterrows():
        master_count=master_count+1;
    print(master_count)

    Attendance_count=0
    directmatch=0
    cosineL=[]
    cosinematch=0
    count_levenshtein_match=0
    bas_name = os.path.basename(fileName)
    split_name = bas_name.split(".")[0]
    successListWorksheet.write(nameCount, 0, split_name)
    failuresListWorksheet.write(nameCountF, 0, split_name)
    nameCount = nameCount+1
    nameCountF = nameCountF+1
    countF = 0
    for i, row in AL.iterrows():
        Attendance_count+=1
        Attendeename=f"{row['Attendee']}"
        upper_case_AN=Attendeename.upper()
        sorted_Attendeename= ''.join(sorted(upper_case_AN))
        ans=900
        flag=0
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            if(Mastername==upper_case_AN):
                flag=1
                break
            
        if(flag==1):
            directmatch=directmatch+1
        else:
            cosineL.append(Attendeename)
            count_levenshtein_match+=1
    #print(cosineL, attencount, directmatch )

    levenshtein_threshold_0=0
    levenshtein_threshold_1=0
    levenshtein_remaining=0
    for i in cosineL:
        attendeename=i
        ans=500
        upper_case_attendeename=attendeename.upper()
        upper_case_removed=upper_case_attendeename.replace(" ", "")
        sorted_attendeename= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            Scholar_number = f"{row2['Registration Number']}"
            master_name_removed=Mastername.replace(" ", "")
            sorted_mastername= ''.join(sorted(master_name_removed))
            cos= (enchant.utils.levenshtein(str(sorted_attendeename), str(sorted_mastername)))
            if(cos<=ans):
                ans=cos
                resultant_mastername= Mastername
                resultant_scholarnumber= Scholar_number
                resultant_master_name=sorted_mastername
        print(sorted_attendeename+"  "+resultant_master_name+"  "+attendeename+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==0):
            levenshtein_threshold_0+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        elif(ans==1):
            levenshtein_threshold_1+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        
        else:
            levenshtein_remaining+=1
            failuresListWorksheet.write(nameCountF, 0, attendeename)
            countF = countF+1
            nameCountF = nameCountF+1
            
    if countF == 0:
        failuresListWorksheet.write(nameCountF, 0, "No Failures")
        nameCountF = nameCountF+1
        
    #print(cosineS1,cosineS2,cosineF)
    nameCount = nameCount+1 
    nameCountF = nameCountF+1
            
     
    master_count
    x=Attendance_count/master_count
    y=x*100
    total_per=round(y,2)
    directmatch
    total_per
    per_dir_str_match=directmatch/Attendance_count 
    per_dir_str_match=per_dir_str_match*100
    per_dir_str_match=round(per_dir_str_match,2)
    lev_str_match=count_levenshtein_match/Attendance_count
    lev_str_match=lev_str_match*100
    lev_str_match=round(lev_str_match,2)
    lev_per_thr_0=levenshtein_threshold_0/Attendance_count
    lev_per_thr_0=lev_per_thr_0*100
    lev_per_thr_0=round(lev_per_thr_0,2)
    lev_per_thr_0
    lev_per_thr_1=levenshtein_threshold_1/Attendance_count
    lev_per_thr_1=lev_per_thr_1*100
    lev_per_thr_1=round(lev_per_thr_1,2)
    lev_per_thr_rem=levenshtein_remaining/Attendance_count
    lev_per_thr_rem=lev_per_thr_rem*100
    lev_per_thr_rem=round(lev_per_thr_rem,2)
            
    worksheet.write(0,0,'Date')
    worksheet.write(0, 1, 'Total percentage of attendee')
    worksheet.write(0, 2, 'Total percentage of direct string match' )
    worksheet.write(0, 3, 'Total percentage of levenshtein match' )
    worksheet.write(0, 4, 'percentage of levenshtein match with threshold as 0' )
    worksheet.write(0, 5, 'percentage of levenshtein match with threshold as 1' )
    worksheet.write(0, 6, 'Failure percentage(percentage of remaining levenshtein match)' )
    worksheet.write(0, 7, 'Number of Attendee')
    worksheet.write(0, 8, 'Number of direct string match')
    worksheet.write(0, 9, 'Number of levenshtein string match')
    worksheet.write(0, 10, 'Number of levenshtein match-0')
    worksheet.write(0, 11, 'Number of levenshtein match-1')
    worksheet.write(0, 12,'Number of levenshtein match-remaining')
    
    date_file = fileName.split(".")
    date_tbu = date_file[0]

    worksheet.write(rowCount,0,date_tbu)
    worksheet.write(rowCount, 1, total_per)
    worksheet.write(rowCount,2, per_dir_str_match )
    worksheet.write(rowCount, 3, lev_str_match)
    worksheet.write(rowCount, 4, lev_per_thr_0)
    worksheet.write(rowCount, 5, lev_per_thr_1)
    worksheet.write(rowCount, 6, lev_per_thr_rem)
    worksheet.write(rowCount, 7, Attendance_count )
    worksheet.write(rowCount, 8, directmatch)
    worksheet.write(rowCount, 9, count_levenshtein_match)
    worksheet.write(rowCount, 10, levenshtein_threshold_0)
    worksheet.write(rowCount, 11, levenshtein_threshold_1)
    worksheet.write(rowCount, 12, levenshtein_remaining)
    return (nameCount, nameCountF)

In [3]:
def checkIfXlsx(fileName):
    fileData = fileName.split(".")
    fileExt = fileData[-1]
    fileBaseName = os.path.basename(fileName)
    if(fileBaseName=='levenshtein_output_set_2.xlsx' or fileBaseName=='MasterStudentList-2020.xlsx' or
        fileBaseName=='Output_Success.xlsx' or fileBaseName=='Output_Failures.xlsx'or fileBaseName=='$stats_final_layer-2_final.xlsx'):
        return False
    if(fileExt=='xlsx'):
        return True
    return False

if __name__ == '__main__':
    ML = pd.read_excel("MasterStudentList-2020.xlsx")
    workbook = xlsxwriter.Workbook("levenshtein_output_set_2.xlsx",)
    successListWorkbook = xlsxwriter.Workbook("Output_Success.xlsx")
    failuresListWorkbook = xlsxwriter.Workbook("Output_Failures.xlsx")
    worksheet = workbook.add_worksheet()
    successListWorksheet = successListWorkbook.add_worksheet()
    failuresListWorksheet = failuresListWorkbook.add_worksheet()
    dir_list = os.listdir()
    fileCount = 0
    nameCount = 0
    nameCountF = 0
    for file in dir_list:
        if(checkIfXlsx(fileName=file)):
            print(file)
            fileCount = fileCount+1
            (camCount, camCount_f) = read_excel(file, fileCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF)
            nameCount = camCount
            nameCountF = camCount_f
    workbook.close()
    successListWorkbook.close()
    failuresListWorkbook.close()

2022-01-10-C2.xlsx
                     Attendee Score     1:46 PM         1:55 PM
6             ADAPAKALA AKASH   2/2           ✅               ✅
7                 ASISH PADHI   2/2           ✅               ✅
8               BODALA RAJESH   2/2           ✅               ✅
9   BYRRAJU SATISH KUMAR RAJU   2/2           ✅               ✅
10             Chandan Mishra   2/2           ✅               ✅
11          DUKKA TEJEPRAKASH   2/2           ✅               ✅
12         Gedela Devi Prasad   2/2           ✅               ✅
13          GOUTAM KUMAR DASH   2/2           ✅               ✅
14            HARSHITA MISHRA   2/2           ✅               ✅
15        INDUKURI ANIL VARMA   2/2           ✅               ✅
16                JAMI MUKESH   2/2           ✅               ✅
17    KALINGO AUROBINDO NAYAK   2/2           ✅               ✅
18          KELLA KIRAN KUMAR   2/2           ✅               ✅
19         KENGAM CHIRANJEEVI   2/2           ✅               ✅
20          KODI RAMA

AAIIKNRSS  AAIIKNRSS  S SAI KIRAN   S SAI  KIRAN  0
AAAADHHKNNPRRS  AAAADHHKNNPRRS  Shankar Pradhan   SHANKAR  PRADHAN  0
.AAAADIJLLMPS  .AAAADIJLLMPS  SILLA.PADMAJA   SILLA.PADMAJA    0
AAHIIJPRRSTTY  AAHIIJPRRSTTY  SRIJA TRIPATHY   SRIJA  TRIPATHY  0
AADEHKMNOORSSUUUV  AADEHKMNOORSSUUUV  SUVENDU KUMAR SAHOO   SUVENDU KUMAR  SAHOO  0
AAADGINNRSTUUV  AAADGINNRSTUUV  Tangudu Sravani   TANGUDU  SRAVANI  0
AACEEHHIIMMNRSY  AACEEHHIIMMNRSY  CHINMAYEE MISHRA   CHINMAYEE  MISHRA  0
AAAABCDEEHHJNNRR  AAACDHHIMNNRS  Raj chandan Behera   CHANDAN  MISHRA  7
2022-01-10.xlsx
                     Attendee Score     1:46 PM         1:55 PM      2:01 PM  \
6                ABDUL NASEER   4/4           ✅               ✅            ✅   
7             ADAPAKALA AKASH   4/4           ✅               ✅            ✅   
8                 ASISH PADHI   4/4           ✅               ✅            ✅   
9   BYRRAJU SATISH KUMAR RAJU   4/4           ✅               ✅            ✅   
10             Chandan Mishra 

AABDEELNRSU  AABDEELNRSU  ABDUL NASEER   ABDUL  NASEER  0
AAAAAAADHKKLPS  AAAAAAADHKKLPS  ADAPAKALA AKASH   ADAPAKALA  AKASH  0
AAACDHHIMNNRS  AAACDHHIMNNRS  Chandan Mishra   CHANDAN  MISHRA  0
ADEIIJNPT  ADEIIJNPT  DIPTI JENA   DIPTI  JENA  0
AAADDDEEEGILPRSV  AAADDDEEEGILPRSV  Gedela Devi Prasad   GEDELA  DEVI PRASAD  0
AAAHHHIIMRRSST  AAAHHHIIMRRSST  HARSHITA MISHRA   HARSHITA  MISHRA  0
AAADIIIKLMNNRRUUV  AAADIIIKLMNNRRUUV  INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  0
AEHIJKMMSU  AEHIJKMMSU  JAMI MUKESH   JAMI  MUKESH  0
AAAEIKKKLLMNRRU  AAAEIKKKLLMNRRU  KELLA KIRAN KUMAR   KELLA  KIRAN KUMAR  0
AAADHIIKLNPPR  AAADHIIKLNPPR  LIPIKA PRADHAN   LIPIKA  PRADHAN  0
AAAAADHHIKLNNNPRT  AAAAADHHIKLNNNPRT  NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  0
AIIKMNNRTU  AIIKMNNRTU  Nitin Kumar   NITIN  KUMAR  0
AAAHKKMMOPRRSU  AAAHKKMMOPRRSU  Om Prakash Kumar   OM  PRAKASH KUMAR  0
AAAABDHHINPPRST  AAAABDHHINPPRST  PITABASH PRADHAN   PITABASH  PRADHAN  0
AAIIKNRSS  AAIIKNRSS  S SAI KIRAN   S SAI  K

99
AABDEELNRSU  AABDEELNRSU  ABDUL NASEER   ABDUL  NASEER  0
AAAAAAADHKKLPS  AAAAAAADHKKLPS  ADAPAKALA AKASH   ADAPAKALA  AKASH  0
AAAABHIJJKMRRRRSSTUUUY  AAAABHIJJKMRRRRSSTUUUY  BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  0
AAACDHHIMNNRS  AAACDHHIMNNRS  Chandan Mishra   CHANDAN  MISHRA  0
AACEEHHIIMMNRSY  AACEEHHIIMMNRSY  CHINMAYEE MISHRA   CHINMAYEE  MISHRA  0
ADEIIJNPT  ADEIIJNPT  DIPTI JENA   DIPTI  JENA  0
AAADEEHJKKKPRSTU  AAAADEHJKKKPRSTU  DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  2
AAADDDEEEGILPRSV  AAADDDEEEGILPRSV  Gedela Devi Prasad   GEDELA  DEVI PRASAD  0
AAAHHHIIMRRSST  AAAHHHIIMRRSST  HARSHITA MISHRA   HARSHITA  MISHRA  0
AAADIIIKLMNNRRUUV  AAADIIIKLMNNRRUUV  INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  0
AEHIJKMMSU  AEHIJKMMSU  JAMI MUKESH   JAMI  MUKESH  0
AAAABDGIIKKLNNNOOORUY  AAAABDGIIKKLNNNOOORUY  KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  0
AAAEIKKKLLMNRRU  AAAEIKKKLLMNRRU  KELLA KIRAN KUMAR   KELLA  KIRAN KUMAR  0
AACEEEGHIIJKMNNRV  AACEEEGHIIJK

AADHIIJT  AADHIIJT  ADITI JHA   ADITI  JHA  0
AAEHIMNORSS  AAEHIMNORSS  AMISHA SOREN   AMISHA  SOREN  0
AAHKKMNRSUU  AAHKKMNRSUU  ANKUSH KUMAR   ANKUSH  KUMAR  0
AAACIIKLLMPRT  AAACIIKLLMPRT  ARPITA MALLICK   ARPITA  MALLICK  0
AAABKMNRSTU  AAABKMNRSTU  BASANT KUMAR   BASANT  KUMAR  0
AAAAADHHIMPRRSUVV  AAAAADHHIMPRRSUVV  DHARMAVARAPU SHIVA   DHARMAVARAPU  SHIVA  0
AAACDHLNOR  AAACDHLNOR  DOLA CHARAN   DOLA  CHARAN  0
AAAAAGHKMMMRRSTUU  AAAAAGHKMMMRRSTUU  GAUTAM KUMAR SHARMA   GAUTAM KUMAR  SHARMA  0
AAGKLMOPRU  AAGKLMOPRU  GOPAL KUMAR   GOPAL  KUMAR  0
.AAAHKKMNRSU  AAHKKMNRSUU  K. UMA SHANKAR   ANKUSH  KUMAR  3
ABHHIKKMOORSUU  ABHHIKKMOORSUU  KHUSHBOO KUMARI   KHUSHBOO  KUMARI  0
AAHIKMMNRSU  AAHIKMMNRSU  MANISH KUMAR   MANISH  KUMAR  0
AAAADDHHIIKLMNNSSU  AAAADDHHIIKLMNNSSU  MD AADIL HUSSAIN KHAN   MD AADILHUSSAIN KHAN  0
AAADDLMMMOOQS  AAADDLMMMOOQS  MD MAQSOOD ALAM   MD  MAQSOOD ALAM  0
AAEEKMNNRTUV  AAEEKMNNRTUV  NAVNEET KUMAR   NAVNEET  KUMAR  0
AAIILMPRSUY  AAIILMPRSUY  PISYA M

...

AABDEELNRSU  AABDEELNRSU  ABDUL NASEER   ABDUL  NASEER  0
AAAAAAADHKKLPS  AAAAAAADHKKLPS  ADAPAKALA AKASH   ADAPAKALA  AKASH  0
AAACDHHIMNNRS  AAACDHHIMNNRS  Chandan Mishra   CHANDAN  MISHRA  0
AAAHHHIIMRRSST  AAAHHHIIMRRSST  HARSHITA MISHRA   HARSHITA  MISHRA  0
AAADIIIKLMNNRRUUV  AAADIIIKLMNNRRUUV  INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  0
AAAAAAIIKKLMNNSV  AAAAAAIIKKLMNNSV  KANAKALA SIVAMANI   KANAKALA  SIVAMANI  0
AAADHIIKKMNORRS  AAADHIIKKMNORRS  KODI RAMA KRISHNA   KODI RAMA  KRISHNA  0
AAADHIIKLNPPR  AAADHIIKLNPPR  LIPIKA PRADHAN   LIPIKA  PRADHAN  0
AAAADHILLMMNNSTUV  AAAADHILLMMNNSTUV  MANDAVALLI SUMANTH   MANDAVALLI  SUMANTH  0
AAAAAEGHMNNNPT  AAAAAEGHMNNNPT  MEGHANA PATNANA   MEGHANA  PATNANA  0
AAAAADHHIKLNNNPRT  AAAAADHHIKLNNNPRT  NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  0
AIIKMNNRTU  AIIKMNNRTU  Nitin Kumar   NITIN  KUMAR  0
AAAHKKMMOPRRSU  AAAHKKMMOPRRSU  Om Prakash Kumar   OM  PRAKASH KUMAR  0
AAAAABMNPRTV  AAAAABMNPRRTV  PAVAN BARATAM   BARATRAM  PAVAN  1
AAAABDHH

AIIIKMPRRTTU  AIIIKMPRRTTU  TRIPTI KUMARI   TRIPTI  KUMARI  0
AAADHKNNPRTUU  AAADHKNNPRTUU  Tukuna Pradhan   TUKUNA  PRADHAN  0
AAHIKKMRSUV  AAHIKKMRSUV  VIKASH KUMAR   VIKASH  KUMAR  0
AEHIKLMMRSUV  AEHIKLMMRSUV  VIMLESH KUMAR   VIMLESH  KUMAR  0
AEEIKMNRTUV  AEEIKMNRTUV  VINEET KUMAR   VINEET  KUMAR  0
AADDEEHKLMRRSSUUY  AADDEEHKLMRRSSUUY  YEDDLA SURESH KUMAR   YEDDLA SURESH  KUMAR  0
2022-03-07.xlsx
                     Attendee Score     1:46 PM         1:55 PM      2:01 PM
6                ABDUL NASEER   3/3           ✅               ✅            ✅
7             ADAPAKALA AKASH   3/3           ✅               ✅            ✅
8                 ASISH PADHI   3/3           ✅               ✅            ✅
9               BODALA RAJESH   3/3           ✅               ✅            ✅
10             Chandan Mishra   3/3           ✅               ✅            ✅
11          DUKKA TEJEPRAKASH   2/3           ✅               ✅            ❎
12         Gedela Devi Prasad   3/3           ✅        

AABDEELNRSU  AABDEELNRSU  ABDUL NASEER   ABDUL  NASEER  0
AAAAAAADHKKLPS  AAAAAAADHKKLPS  ADAPAKALA AKASH   ADAPAKALA  AKASH  0
AADHIIJT  AADHIIJT  ADITI JHA   ADITI  JHA  0
AAEHIMNORSS  AAEHIMNORSS  AMISHA SOREN   AMISHA  SOREN  0
AAACIIKLLMPRT  AAACIIKLLMPRT  ARPITA MALLICK   ARPITA  MALLICK  0
AADHHIIPSS  AADHHIIPSS  ASISH PADHI   ASISH  PADHI  0
AABHIIKMRRTU  AABHIIKMRRTU  BHARTI KUMARI   BHARTI  KUMARI  0
AAABDEHJLORS  AAABDEHJLORS  BODALA RAJESH   BODALA  RAJESH  0
AAAABHIJJKMRRRRSSTUUUY  AAAABHIJJKMRRRRSSTUUUY  BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  0
AACEEHHIIMMNRSY  AACEEHHIIMMNRSY  CHINMAYEE MISHRA   CHINMAYEE  MISHRA  0
AAAAADHHIMPRRSUVV  AAAAADHHIMPRRSUVV  DHARMAVARAPU SHIVA   DHARMAVARAPU  SHIVA  0
ADEIIJNPT  ADEIIJNPT  DIPTI JENA   DIPTI  JENA  0
AAACDHLNOR  AAACDHLNOR  DOLA CHARAN   DOLA  CHARAN  0
AAADEEHJKKKPRSTU  AAAADEHJKKKPRSTU  DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  2
AAADDDEEEGILPRSV  AAADDDEEEGILPRSV  Gedela Devi Prasad   GEDELA  DEVI PRASAD  0


AAAAADHHIKLNNNPRT  AAAAADHHIKLNNNPRT  NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  0
AAAHKKMMOPRRSU  AAAHKKMMOPRRSU  Om Prakash Kumar   OM  PRAKASH KUMAR  0
AAAAABMNPRTV  AAAAABMNPRRTV  PAVAN BARATAM   BARATRAM  PAVAN  1
AAAAAABEEEJKLNNPR  AAAAAABEEEJKLNNPR  Penabakala Neeraja   PENABAKALA  NEERAJA  0
AAIILMPRSUY  AAIILMPRSUY  PISYA MURALI   PISYA  MURALI  0
AAAABDHHINPPRST  AAAABDHHINPPRST  PITABASH PRADHAN   PITABASH  PRADHAN  0
AAHKLMRRUU  AAHKLMRRUU  RAHUL KUMAR   RAHUL  KUMAR  0
AAABEEHIJRTV  AAABEEHIJRTV  RAJEEV BAITHA   RAJEEV  BAITHA  0
AAEGHIIKMNORRSSTU  AAEGHIIKMORRSSTU  RITESH KUMAR GOSAIN   RITESH KUMAR GOSAI  1
AJMNOS  AIKLMNRSUU  S MANOJ   SUNIL  KUMAR  6
ADDEEHHIIMRSSUU  ADDEEHHIIMRSSUU  SHIREESHA IDDUMU   SHIREESHA  IDDUMU  0
.AAAADIJLLMPS  .AAAADIJLLMPS  SILLA.PADMAJA   SILLA.PADMAJA    0
AAEEGHIRRSSUU  AAEEGHIRRSSUU  SIREESHA AGURU   AGURU  SIREESHA  0
AAAEEGINNORSSTT  AAAEEGINNORSSTT  SONTANA GEETASRI   SONTANA  GEETASRI  0
AAAAAADHHHINPRRRSSUV  AAAAAADHHHINPRRRSSUV  SR

AAEGHIIKMNORRSSTU  AAEGHIIKMORRSSTU  RITESH KUMAR GOSAIN   RITESH KUMAR GOSAI  1
AJMNOS  AIKLMNRSUU  S MANOJ   SUNIL  KUMAR  6
AAIIKLMNORSU  AAIIKLMNORSU  SALONI KUMARI   SALONI  KUMARI  0
AAAAADHHIKKNPRSST  AAAAADHHIKKNPRSST  SHAKTI PRASAD NAHAK   SHAKTI  PRASAD NAHAK  0
AAABDHHILMNSSUY  AAABDHHILMNSSUY  SHUBHAM SANDILYA   SHUBHAM  SANDILYA  0
.AAAADIJLLMPS  .AAAADIJLLMPS  SILLA.PADMAJA   SILLA.PADMAJA    0
AAEEGIMNNNPRSUUV  AAEEGIMNNNPRSUUV  SINGUPURAM NAVEEN   SINGUPURAM  NAVEEN  0
AAEEGHIRRSSUU  AAEEGHIRRSSUU  SIREESHA AGURU   AGURU  SIREESHA  0
AAAEEGINNORSSTT  AAAEEGINNORSSTT  SONTANA GEETASRI   SONTANA  GEETASRI  0
AAAAAADHHHINPRRRSSUV  AAAAAADHHHINPRRRSSUV  SRI HARSHA ANDHAVARAPU   ANDHAVARAPU  SRI HARSHA  0
AAHIIJPRRSTTY  AAHIIJPRRSTTY  SRIJA TRIPATHY   SRIJA  TRIPATHY  0
AIKLMNRSUU  AIKLMNRSUU  SUNIL KUMAR   SUNIL  KUMAR  0
AAEHHHKNRSSSTU  AAEHHHKNRSSSTU  SUSHANT SHEKHAR   SUSHANT  SHEKHAR  0
AADEHKMNOORSSUUUV  AADEHKMNOORSSUUUV  SUVENDU KUMAR SAHOO   SUVENDU KUMAR  SAHOO  0
